# ...in C

## World Bank API in C

by Cyrus Hartwin Gomes and Dr. Vincent Scalfani

See the [World Bank API documentation](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation)

These recipe examples were tested on July 25, 2023

### Setup

First, install the CURL and jq package by typing the following command in the terminal:

In [ ]:
!sudo apt install curl jq libcurl4-openssl-dev

First we set a directory where we want the WorldBank directory for our projects to be created 

In [1]:
!mkdir World_Bank

We change the directory to the folder we created

In [ ]:
%cd World_Bank


### 1. Get list of country iso2Codes and names

For obtaining data from the World Bank API, it is helpful to first obtain a list of country codes and names.

We initialize a folder for the current project that we are working on. And then change to that directory

In [3]:
!mkdir iso2Codes_names

In [ ]:
%cd iso2Codes_names

Then we utilize the following command (%%file) to create the following makefile which will compile our program and create an executable.

In [ ]:
%%file makefile

#sets the variable CC to gcc, which is used to build the program
CC=gcc

#enable debugging information and enable all compiler warnings
CFLAGS=-g -Wall

#sets the bin variable as the name of the binary file we are creating
BIN=iso2Codes

#creates the binary file with the name we put
all: $(BIN)

#maps any file ending in .c to a binary executable. 
#"$<" represents the .c file and "$@" represents the target binary executable
%: %.c

#compiles the .c file using the gcc compiler with the CFLAGS and links 
#resulting binary with the CURL library
	$(CC) $(CFLAGS) $< -o $@ -lcurl

#clean target which removes specific files
clean:

#removes the binary file and an ".dSYM" (debug symbols for debugging) directories
#the RM command used -r to remove directories and -f to force delete
	$(RM) -rf $(BIN) *.dSYM


The command is used again to create our .c file which contains the code for the program

In [ ]:
%%file iso2Codes.c

#include <curl/curl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/*CURL program that retrieves JSON data from the World Bank API*/

int main (int argc, char* argv[]){
    
    //if arguments are invalid just return
    if (argc > 1){                                                                                      
        printf("Error. Please try again correctly.\n");
        return -1;
    }

    //initializes the CURL HTTP connection
    CURL *curl = curl_easy_init();

    //bits of the url that are joined together later                                                                      
    char api[] = "https://api.worldbank.org/v2/";                            
    char url[1000];
    char label[] = "country/?format=json&per_page=500";

    //checks if CURL initialization is a success or not
    if (!curl){                                                                                         
        fprintf(stderr, "init failed\n");
        return EXIT_FAILURE;
    }
        
    //combines all the bits to produce a functioning url
    sprintf(url, "%s%s", api, label);                                             
                                          

    //sets the url to which the HTTP request will be sent to
    //first parameter is for the initialized curl HTTP request, second for the option to be set, and third for the value to be set
    curl_easy_setopt(curl, CURLOPT_URL, url);

    //if result is not retrieved then output error
    CURLcode result = curl_easy_perform(curl);

    //if result is not retrieved then output error
    if (result != CURLE_OK){                                                                            
        fprintf(stderr, "download problem: %s\n", curl_easy_strerror(result));
    }

    //deallocates memory for the CURL connection
    curl_easy_cleanup(curl);                                                                            
    return EXIT_SUCCESS;
}

The folowing program is run, and an executable is created after using the following command:

In [ ]:
!make

To output the data from the WOrld Bank API, we enter the following command:

In [8]:
!./iso2Codes | jq '.[1][0]'

{
  "id": "ABW",
  "iso2Code": "AW",
  "name": "Aruba",
  "region": {
    "id": "LCN",
    "iso2code": "ZJ",
    "value": "Latin America & Caribbean "
  },
  "adminregion": {
    "id": "",
    "iso2code": "",
    "value": ""
  },
  "incomeLevel": {
    "id": "HIC",
    "iso2code": "XD",
    "value": "High income"
  },
  "lendingType": {
    "id": "LNX",
    "iso2code": "XX",
    "value": "Not classified"
  },
  "capitalCity": "Oranjestad",
  "longitude": "-70.0167",
  "latitude": "12.5167"
}


In [ ]:
!./iso2Codes | jq '.[1][] | .iso2Code + ": " + .name'

### 2. Compile a Custom Indicator Dataset

There are many availabe indicators: https://data.worldbank.org/indicator

We wll select either of the three indicators for this example:

1. Scientific and Technical Journal Article Data = [IP.JRN.ARTC.SC](https://data.worldbank.org/indicator/IP.JRN.ARTC.SC?view=chart)

2. Patent Applications, residents = [IP.PAT.RESD](https://data.worldbank.org/indicator/IP.PAT.RESD?view=chart)

3. GDP per capita (current US$) Code = [NY.GDP.PCAP.CD](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?view=chart)

Note that these three selected indictaors have a [CC-BY 4.0 license](https://datacatalog.worldbank.org/public-licenses#cc-by) We will compile this indicator data for the United States (US) and United Kingdom (GB)




We change the directory of the World_Bank folder to create a new one for our project

In [ ]:
%cd ..

We set a directory where we want the WorldBank directory for the custom indicator project 

In [11]:
!mkdir Custom_indicator

Then we change the directory to the folder we created

In [ ]:
%cd Custom_indicator

Then we utilize the following command (%%file) to create the following makefile which will compile our program and create an executable.

In [ ]:
%%file makefile

#sets the variable CC to gcc, which is used to build the program
CC=gcc

#enable debugging information and enable all compiler warnings
CFLAGS=-g -Wall

#sets the bin variable as the name of the binary file we are creating
BIN=custom_indicator

#creates the binary file with the name we put
all: $(BIN)

#maps any file ending in .c to a binary executable. 
#"$<" represents the .c file and "$@" represents the target binary executable
%: %.c

#compiles the .c file using the gcc compiler with the CFLAGS and links 
#resulting binary with the CURL library
	$(CC) $(CFLAGS) $< -o $@ -lcurl

#clean target which removes specific files
clean:

#removes the binary file and an ".dSYM" (debug symbols for debugging) directories
#the RM command used -r to remove directories and -f to force delete
	$(RM) -rf $(BIN) *.dSYM


The command is used again to create our .c file which contains the code for the program

In [ ]:
%%file custom_indicator.c

#include <curl/curl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/*CURL program that retrieves JSON data from the World Bank API
This program allows custom indicator data set to be used along with the country specified*/


/* We are going to be inputting the custom indicator and country like this: ./custom_indicator -c "UK" -i "IP.JRN.ARTC.SC"
If the arguments are missing then we use the default: "US" "IP.JRN.ARTC.SC"
*/

int main (int argc, char* argv[]){
    
    //if arguments are invalid just return
    if (argc > 5){                                                                                      
        printf("Error. Please try again correctly.\n");
        return -1;
    }

    //default country and indicator codes
    char country[10] = {};
    char indicator[100] = {}; 

    //if there is ./custom_indiactor -c/-i
    if ((argc == 1) || ((argc == 2) && ((strcmp(argv[1], "-c")==0) || (strcmp(argv[1], "-i")==0)))){
        //these arguments run the default parameters and keeps the codes as they are
        strcat(country,"US");
        strcat(indicator, "IP.JRN.ARTC.SC");
    }

    //if there is ./custom indicator -c "UK"
    else if ((argc == 3) && (strcmp(argv[1], "-c")==0)){
        //only the country code is changed
        strcat(country,argv[2]);
        strcat(indicator, "IP.JRN.ARTC.SC");
    }

    //if there is ./custom indicator -c "UK" -i
    else if ((argc == 4) && (strcmp(argv[1], "-c")==0) && (strcmp(argv[3], "-i")==0)){
        //only the country code is changed
        strcat(country,argv[2]);
        strcat(indicator, "IP.JRN.ARTC.SC");
    }

    //if there is ./custom indicator -c "UK" -i "IP.JRN.ARTC.SC"
    else if ((argc == 5) && (strcmp(argv[1], "-c")==0) && (strcmp(argv[3], "-i")==0)){
        //both the country and indicator codes are changed
        strcat(country,argv[2]);
        strcat(indicator, argv[4]);
    }

    //if there is ./custom indicator -i "IP.JRN.ARTC.SC"
    else if ((argc == 3) && (strcmp(argv[1], "-i")==0)){
        //only the indicator code is changed
        strcat(country,"US");
        strcat(indicator, argv[2]);
    }

    //if there is ./custom indicator -i "IP.JRN.ARTC.SC" -c
    else if ((argc == 4) && (strcmp(argv[1], "-i")==0) && (strcmp(argv[3], "-c")==0)){
        //only the indicator code is changed
        strcat(country,"US");
        strcat(indicator, argv[2]);
    }

    //if there is ./custom indicator -i "IP.JRN.ARTC.SC" -c "UK" 
    else if ((argc == 5) && (strcmp(argv[1], "-i")==0) && (strcmp(argv[3], "-c")==0)){
        //both the indicator and country codes are changed
        strcat(country,argv[4]);
        strcat(indicator, argv[2]);
    }

    else{
        printf("usage: ./custom_indicator [-i] indicator [-c] country\n\n");
        printf("the custom_indicator program is used to retrieve json data from the World Bank API\n\n");
        printf("optional arguments\n");
        printf("\t -i indicator    optional custom indicator code; default is 'IP.JRN.ARTC.SC', see: https://data.worldbank.org/indicator\n");
        printf("\t -c country      optional custom country code; default is 'US'\n");
        return -1;
    }

    //initializes the CURL HTTP connection
    CURL *curl = curl_easy_init();

    //bits of the url that are joined together later
    char api[] = "https://api.worldbank.org/v2/";                                                                     
    char type1[] = "country/";
    char type2[] = "/indicator/";                           
    char url[1000];
    char label[] = "/?format=json&per_page=500";

    //checks if CURL initialization is a success or not
    if (!curl){                                                                                         
        fprintf(stderr, "init failed\n");
        return EXIT_FAILURE;
    }
        
    //combines all the bits to produce a functioning url
    sprintf(url, "%s%s%s%s%s%s", api, type1 , country, type2, indicator, label);                                             
                                          

    //sets the url to which the HTTP request will be sent to
    //first parameter is for the initialized curl HTTP request, second for the option to be set, and third for the value to be set
    curl_easy_setopt(curl, CURLOPT_URL, url);

    //if result is not retrieved then output error
    CURLcode result = curl_easy_perform(curl);

    //if result is not retrieved then output error
    if (result != CURLE_OK){                                                                            
        fprintf(stderr, "download problem: %s\n", curl_easy_strerror(result));
    }

    //deallocates memory for the CURL connection
    curl_easy_cleanup(curl);                                                                            
    return EXIT_SUCCESS;
}

In [ ]:
!make

To output the data from the World Bank API with the default indicator and country (IP.JRN.ARTC.SC, US), we enter the following command:

In [16]:
!./custom_indicator | jq '.[1][] | "\(.date)':' \(.value)"' | head -n 10

"2022: null"
"2021: null"
"2020: 455855.57"
"2019: 438020.45"
"2018: 435033.88"
"2017: 430198.17"
"2016: 428476.45"
"2015: 428203.78"
"2014: 428140.59"
"2013: 426596.66"


To output the data from the World Bank API with the custom indicator and country, we enter the following command:

In [17]:
!./custom_indicator -i 'NY.GDP.PCAP.CD' -c 'GB' | jq '.[1][] | "\(.date)':' \(.value)"' | head -n 10

"2022: 45850.4261222629"
"2021: 46585.8975644567"
"2020: 40318.4169225055"
"2019: 42747.080460496"
"2018: 43306.3083049317"
"2017: 40622.6893883232"
"2016: 41146.0773555246"
"2015: 45071.0743234873"
"2014: 47447.5889322667"
"2013: 43449.0917173139"


To compile a custom indicator dataset we need to have parse the data into a single .txt file

We enter the following command, to output first indicator data to the .txt file:

In [18]:
!./custom_indicator -i 'NY.GDP.PCAP.CD' -c 'US' | jq '.[1][] | "\(.date)':' \(.value)"' > output1.txt

We do the same for the 2nd and the 3rd indicator data:

In [19]:
!./custom_indicator -i 'IP.JRN.ARTC.SC' -c 'US' | jq '.[1][] | "\(.date)':' \(.value)"' > output2.txt

In [20]:
!./custom_indicator -i 'IP.PAT.RESD' -c 'US' | jq '.[1][] | "\(.date)':' \(.value)"' > output3.txt

We then join the data together in different columns and output it to the specified .txt file:

In [21]:
!join output1.txt output2.txt > output12.txt

The same is done to create the final combined data set:

In [22]:
!join output12.txt output3.txt > output_combined.txt

We then remove the """ and ":" to create the custom data set

In [23]:
!cat output_combined.txt | tr -d '",:' | head -5

2022 76398.5917422054 null null
2021 70219.472454115 null null
2020 63528.6343027508 455855.57 269586
2019 65120.3946628653 438020.45 285113
2018 62823.309438197 435033.88 285095
